# 动手学深度学习 章节8.2 文本预处理

In [25]:
import re
from typing import List, Union	# type: ignore
from collections import Counter

from d2l import torch as d2l

# 读取数据

time-machine小说

In [8]:
d2l.DATA_HUB['time_machine'] = (d2l.DATA_URL + 'timemachine.txt', '090b5e7e70c295757f55df93cb0a180b9691891a')

def read_time_machine():  #@save
    """将时间机器数据集加载到文本行的列表中"""
    with open(d2l.download('time_machine', cache_dir="./dataset"), 'r') as f:
        lines = f.readlines()
    return [re.sub('[^A-Za-z]+', ' ', line).strip().lower() for line in lines]

In [9]:
lines = read_time_machine()
print(f'# 文本总行数: {len(lines)}')
print(lines[0])
print(lines[10])
print(type(lines))

# 文本总行数: 3221
the time machine by h g wells
twinkled and his usually pale face was flushed and animated the
<class 'list'>


# 词元化

tokenization

两种词元化的形式 (单词, 字符)

In [52]:
def tokenize(line: List[str], token="word"):
	if token == "word":
		return [line.split() for line in lines]
	else:
		return [list(line) for line in lines]

In [53]:
tokens = tokenize(lines)
tokens

[['the', 'time', 'machine', 'by', 'h', 'g', 'wells'],
 [],
 [],
 [],
 [],
 ['i'],
 [],
 [],
 ['the',
  'time',
  'traveller',
  'for',
  'so',
  'it',
  'will',
  'be',
  'convenient',
  'to',
  'speak',
  'of',
  'him'],
 ['was',
  'expounding',
  'a',
  'recondite',
  'matter',
  'to',
  'us',
  'his',
  'grey',
  'eyes',
  'shone',
  'and'],
 ['twinkled',
  'and',
  'his',
  'usually',
  'pale',
  'face',
  'was',
  'flushed',
  'and',
  'animated',
  'the'],
 ['fire',
  'burned',
  'brightly',
  'and',
  'the',
  'soft',
  'radiance',
  'of',
  'the',
  'incandescent'],
 ['lights',
  'in',
  'the',
  'lilies',
  'of',
  'silver',
  'caught',
  'the',
  'bubbles',
  'that',
  'flashed',
  'and'],
 ['passed',
  'in',
  'our',
  'glasses',
  'our',
  'chairs',
  'being',
  'his',
  'patents',
  'embraced',
  'and'],
 ['caressed',
  'us',
  'rather',
  'than',
  'submitted',
  'to',
  'be',
  'sat',
  'upon',
  'and',
  'there',
  'was',
  'that'],
 ['luxurious',
  'after',
  'dinner',

# 根据token出现的频率构建vocab

In [54]:
def count_corpus(tokens):
	if len(tokens) == 0 or isinstance(tokens[0], list):
		tokens = [token for line in tokens for token in line]
	return Counter(tokens)

In [55]:
class Vocab:
	def __init__(self, tokens=None, min_freq=0, reserved_tokens=None) -> None:
		if tokens is None:
			tokens = []
		if reserved_tokens is None:
			reserved_tokens = []
		counter = count_corpus(tokens)
		self._tokens_freqs = sorted(counter.items(), key=lambda x: x[1], reverse=True)
		# 根据index取出token
		self.idx_to_token = ['<unk>'] + reserved_tokens
		# 映射, token -> index
		self.token_to_idx = {tokens: idx for idx, tokens in enumerate(self.idx_to_token)}

		for token, freq in self._tokens_freqs:
			if freq < min_freq:
				break
			if token not in self.token_to_idx:
				self.idx_to_token.append(token)
				self.token_to_idx[token] = len(self.idx_to_token) - 1


	def __len__(self):
		return len(self.idx_to_token)
	

	def __getitem__(self, tokens):
		"""
		给定一个tokens列表, 返回这些token所对应的index
		"""
		if not isinstance(tokens, (list, tuple)):
			return self.token_to_idx.get(tokens, self.unk)
		else:
			return [self.__getitem__(token) for token in tokens]


	def to_tokens(self, indices):
		if not isinstance(indices, (list, tuple)):
			return self.idx_to_token[indices]
		else:
			return [self.idx_to_token[index] for index in indices]
	
	
	@property
	def unk(self):
		return self.token_to_idx.get('<unk>', 0)
	
	
	@property
	def token_freqs(self):
		return self._tokens_freqs


	@property
	def vocab(self):
		return list(self.token_to_idx.items())

In [56]:
vocab = Vocab(tokens)
print(vocab.vocab[:10])

[('<unk>', 0), ('the', 1), ('i', 2), ('and', 3), ('of', 4), ('a', 5), ('to', 6), ('was', 7), ('in', 8), ('that', 9)]


# 对数据集进行预处理

In [60]:
def load_corpus_time_machine(max_tokens=-1):
	line = read_time_machine()
	tokens = tokenize(line, 'char')
	vocab = Vocab(tokens)
	corpus = [vocab[token] for line in tokens for token in line]
	if max_tokens > 0:
		corpus = corpus[:max_tokens]
	
	return corpus, vocab

In [63]:
corpus, vocab = load_corpus_time_machine()